# **Long Term Memory**

In [6]:
import os, dotenv

dotenv.load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

## Setup

In [95]:
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Set up store and memory saver
store = InMemoryStore(
    index={
        "dims": 1024,
        "embed": "cohere:embed-multilingual-v3.0",
    }
) #

In [111]:
tool = create_manage_memory_tool(namespace="memories", instructions="Custom Instructions", store=store)

In [112]:
tool.description

'Create, update, or delete persistent MEMORIES to persist across conversations.\nInclude the MEMORY ID when updating or deleting a MEMORY. Omit when creating a new MEMORY - it will be created for you.\nCustom Instructions'

In [114]:
tool.tool_call_schema.model_json_schema()

{'description': 'Create, update, or delete persistent MEMORIES to persist across conversations.\nInclude the MEMORY ID when updating or deleting a MEMORY. Omit when creating a new MEMORY - it will be created for you.\nCustom Instructions',
 'properties': {'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Content'},
  'action': {'default': 'create',
   'enum': ['create', 'update', 'delete'],
   'title': 'Action',
   'type': 'string'},
  'id': {'anyOf': [{'format': 'uuid', 'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Id'}},
 'title': 'manage_memory',
 'type': 'object'}

## ReAct Agent with Memory tools

In [101]:
# Create agent with memory tools
agent = create_react_agent(
    "google_genai:gemini-2.0-flash",
    tools=[
        # Configure memory tools with runtime namespace 
        create_manage_memory_tool(namespace=("memories", "216")),
        create_search_memory_tool(namespace=("memories", "216")),
    ],
    store=store,
)

In [102]:
agent.invoke({"messages": [{"role": "user", "content": "I am Aasher, remember it. And I like Pizza"}]})

{'messages': [HumanMessage(content='I am Aasher, remember it. And I like Pizza', additional_kwargs={}, response_metadata={}, id='e07a4913-8bf7-4161-9816-94c2a0c0b5f1'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'manage_memory', 'arguments': '{"content": "The user is Aasher and he likes Pizza.", "action": "create"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-bf3376dd-b422-4f1c-82aa-368bcb57ff23-0', tool_calls=[{'name': 'manage_memory', 'args': {'content': 'The user is Aasher and he likes Pizza.', 'action': 'create'}, 'id': '7e309065-f51c-42eb-b25d-626f662dd40c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 144, 'output_tokens': 16, 'total_tokens': 160, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='created memory ee414730-2070-43b3-b940-ec2d80bdb373', name='manage_memory', id='e1d1c980-43aa-4ece-acc1-ca5b3ab5710b', tool_call_id='7e3090

In [104]:
agent.invoke({"messages": [{"role": "user", "content": "I am an Astrophile.."}]})

{'messages': [HumanMessage(content='I am an Astrophile..', additional_kwargs={}, response_metadata={}, id='84d078e4-785e-42f9-87e4-430e2d9d5236'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'manage_memory', 'arguments': '{"content": "The user is an Astrophile.", "action": "create"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-17b4a524-7e38-4b43-b309-7c56d48fe144-0', tool_calls=[{'name': 'manage_memory', 'args': {'content': 'The user is an Astrophile.', 'action': 'create'}, 'id': '6750de01-45a0-4975-933f-00d34addda88', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 14, 'total_tokens': 153, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='created memory 7ff79586-914b-4c70-bb28-2a6850fe94c2', name='manage_memory', id='291cf40b-bc51-4ea4-8388-8948f1998f4a', tool_call_id='6750de01-45a0-4975-933f-00d34addda88'),
  AIMessage(

## Searching Memories by namespace

In [105]:
from rich import print

print(store.search(
    ("memories",)
))

[
    Item(namespace=['memories', '216'], key='ee414730-2070-43b3-b940-ec2d80bdb373', value={'content': 'The user is 
Aasher and he likes Pizza.'}, created_at='2025-02-15T14:51:45.756743+00:00', 
updated_at='2025-02-15T14:51:45.756743+00:00', score=None),
    Item(namespace=['memories', '216'], key='7ff79586-914b-4c70-bb28-2a6850fe94c2', value={'content': 'The user is 
an Astrophile.'}, created_at='2025-02-15T14:52:44.806651+00:00', updated_at='2025-02-15T14:52:44.806651+00:00', 
score=None)
]

## Semantic Search

In [107]:
# Search for semantically similar documents
store.search(
    ("memories", "216"),
    query="What is my name??",
    limit=1)

[Item(namespace=['memories', '216'], key='ee414730-2070-43b3-b940-ec2d80bdb373', value={'content': 'The user is Aasher and he likes Pizza.'}, created_at='2025-02-15T14:51:45.756743+00:00', updated_at='2025-02-15T14:51:45.756743+00:00', score=0.3105525703554466)]